<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/hapt_rf_transformer_hier_smote_resample_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [ ]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Some methods and data which will be used below

In [2]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [3]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [4]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [5]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [6]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [7]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [8]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest
  scaler = get_scaler(xtrain)
  xtrain = scale_data(xtrain,scaler)
  xtest  = scale_data(xtest,scaler)

  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,xtest,ytrain,ytest


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Preparing data, xtrain, ytrain
# Last six classes [7 to 12] has very less weightage in data since they are extra classes added
# , made from original six classes
# so, I took more overlapping in them to get slightly more data

xtrain,ytrain = prepare_data('/content/drive/MyDrive/HAPT Data Set/RawData',128,[0,0,0,0,0,0,0,0,0,0,0,0])

In [11]:
originalx = xtrain
originaly=ytrain

In [ ]:
#xtrain=originalx
#ytrain = originaly

In [12]:
ytrain.shape

(5773,)

In [13]:
xtrain.shape

(5773, 128, 2, 3)

In [ ]:
128*6

768

In [14]:
xtrain = xtrain.reshape(5773,768)


In [15]:
xtrain.shape

(5773, 768)

In [16]:
df = pd.DataFrame(xtrain)
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
59,0.555556,-0.118056,-0.054167,0.613003,-1.423011,-0.106901,0.8125,-0.061111,-0.1375,0.339947,...,-0.277778,-0.955699,-1.295951,-0.612087,1.140278,-0.693056,-0.3875,0.066584,0.821308,-0.56108


In [17]:
ytrain.shape

(5773,)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [18]:
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

In [19]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
df, ytrain = oversample.fit_resample(df, ytrain)

In [20]:
xtrain.shape

(5773, 768)

In [21]:
df.shape

(12228, 768)

In [22]:
ytrain.shape

(12228,)

In [23]:
df=df.to_numpy()
xtrain=df.reshape(12228,128,6)

In [24]:
xtrain.shape

(12228, 128, 6)

In [25]:
xtrain=df.reshape(12228,128,2,3)

In [26]:
xtrain.shape

(12228, 128, 2, 3)

**start SVM**

In [27]:
X_train = pd.read_csv('/content/drive/MyDrive/newdata48features.csv', header=None)
xxx,Y_train = resampleData(originalx,originaly)

In [28]:
X_train.head() # mean ,median , std , variance , root mean square , max, min

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0__median,0__mean,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__absolute_maximum,0__minimum,1__median,1__mean,...,4__absolute_maximum,4__minimum,5__median,5__mean,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__absolute_maximum,5__minimum
1,1.01944450285877,1.0193142942447153,0.0024762469750240646,6.131799081315831e-06,1.01931730204617,1.027777854158295,1.027777854158295,1.0125000434425,-0.1250000020616516,-0.12428385819818219,...,0.02138028293848038,-0.02138028293848038,0.003054326167330146,0.0024673228924712025,0.006133609869028267,3.762117002544095e-05,0.006611267070777992,0.01679879426956177,0.01679879426956177,-0.01404990069568157
2,1.01944450285877,1.0199002330079632,0.0033669412028527372,1.1336293063467436e-05,1.0199057905428133,1.02916674604155,1.02916674604155,1.009722259675992,-0.1263888939449057,-0.12505425802505304,...,0.03451388701796532,-0.03451388701796532,0.00366519158706069,0.00544290474636,0.009781032110216805,9.566858914109219e-05,0.011193471365891834,0.03634648025035858,0.03634648025035858,-0.01435533352196217
3,1.01944450285877,1.0201497995182354,0.002750501804412125,7.565260176074354e-06,1.0201535074278145,1.025000070391787,1.025000070391787,1.011111151559246,-0.1305555574387036,-0.13018663702089006,...,0.02626720629632473,-0.02626720629632473,0.0,0.0003436116994635086,0.009007026194748656,8.112652087288845e-05,0.009013578083807598,0.02718350291252136,0.02718350291252136,-0.02382374368607998
4,1.020833394742025,1.0207791411528349,0.0028706455352721085,8.24060578917769e-06,1.0207831775742136,1.027777854158295,1.027777854158295,1.013888935325754,-0.1263888939449057,-0.1276367229824334,...,0.03390302136540413,-0.03390302136540413,0.005345070967450738,0.006118197258047076,0.012574714847580413,0.00015812345349795927,0.013984126400541938,0.03359758853912354,0.03359758853912354,-0.02443460933864117


In [29]:
Y_train.shape

(12228,)

In [30]:
X_train=X_train.iloc[1: , :]


In [31]:
###########################################################################################
x1train,x1test,y1train,y1test=splitData2(X_train,Y_train,'dynamic')
x2train,x2test,y2train,y2test=splitData2(X_train,Y_train,'static')
x3train,x3test,y3train,y3test=splitData2(X_train,Y_train,'transition')

###########################################################################################
x1train.shape,y1train.shape,x3train.shape,y3train.shape

((2139, 48), (2139,), (4279, 48), (4279,))

In [32]:
X_train = np.concatenate([x1train,x2train, x3train], axis=0)
X_test = np.concatenate([x1test,x2test, x3test], axis=0)

Y_train = np.concatenate([y1train,y2train, y3train], axis=0)
Y_test = np.concatenate([y1test,y2test, y3test], axis=0)



In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size = 0.3,random_state=42)


In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((8557, 48), (3671, 48), (8557,), (3671,))

In [78]:
X_test.shape,Y_test.shape

((3671, 48), (3671,))

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, accuracy_score


svc = LinearSVC(random_state=42, tol=1e-5, verbose=1, max_iter=10000)
svc.fit(X_train, Y_train)

svm_pred = svc.predict(X_test)
SVC_pred_test = accuracy_score(Y_test, svm_pred)*100
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, svm_pred)*100))

[LibLinear]Accuracy score TEST: 90.902


In [ ]:
svm_pred.shape

(3671,)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100, 200]
scores=[]
for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(x2train, y2train)
    rf_pred = rf.predict(x2test)
    scores.append(accuracy_score(y2test, rf_pred))
    print('Accuracy score of static activities TEST: {:.3f}'.format(accuracy_score(y2test, rf_pred)*100))

Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.534
Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.752


In [77]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100, 200]
scores=[]
for n_trees in tree_list:

    rf2 = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf2.fit(x3train, y3train)
    rf2_pred = rf2.predict(x3test)
    scores.append(accuracy_score(y3test, rf2_pred))
    print('Accuracy score of transition activities TEST: {:.3f}'.format(accuracy_score(y3test, rf2_pred)*100))

Accuracy score of transition activities TEST: 98.583
Accuracy score of transition activities TEST: 98.638
Accuracy score of transition activities TEST: 98.856
Accuracy score of transition activities TEST: 98.965


**end** **svm**

## Handling missing data and Normalizing data

In [35]:
xtrain,ytrain = remove_null(xtrain,ytrain)

In [36]:
xtrain.shape

(12228, 128, 2, 3)

In [ ]:
2449+2960+364

5773

In [37]:
ytrain.shape

(12228,)

**Hier split and training**

**Dynamic**

In [38]:
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels = splitData(xtrain,ytrain,'dynamic')

In [39]:
dynamicxtrain.shape,dynamicytrain.shape

((2139, 128, 2, 3), (2139, 3))

In [ ]:
####################################################################################################################################

In [40]:
dynamicxtrain.shape,dynamicxtest.shape,dynamicytrain.shape,dynamicytest.shape,dynamiclabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [66]:
#xtrain,ytrain = resampleData(originalx,originaly)
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(xtrain,ytrain,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [43]:
dynamicxtrain = dynamicxtrain.reshape(2139,128,6)
dynamicxtest = dynamicxtest.reshape(918,128,6)

In [70]:
staticxtrain = staticxtrain.reshape(2139,128,6)
staticxtest = staticxtest.reshape(918,128,6)

**start** **transformers**

In [46]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

from matplotlib import pyplot
import matplotlib.pyplot as plt
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [63]:
%cd sample_data

/content/sample_data


In [64]:
import model

learningRate = 3e-4

optimizer = tf.keras.optimizers.Adam(learningRate)
optimizer2 = tf.keras.optimizers.Adam(learningRate)

dynamic_classifier = model.HART((128,6),3)
static_classifier = model.HART((128,6),3)

dynamic_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
static_classifier.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [67]:
from sklearn.utils import class_weight

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(dynamiclabels),
                                                 y = dynamiclabels.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(staticlabels),
                                                 y = staticlabels.ravel())
class_weights2 = {j : temp_weights[j] for j in range(len(temp_weights))}

In [68]:
np.unique(staticlabels)


array([4, 5, 6], dtype=uint32)

In [71]:
checkpoint_filepath = "bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

history0 = dynamic_classifier.fit(
    x=dynamicxtrain,
    y=dynamicytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)

history1 = static_classifier.fit(
    x=staticxtrain,
    y=staticytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights2,
    callbacks=[checkpoint_callback],
)

Epoch 1/100
9/9 [==============================] - ETA: 0s - loss: 0.2946 - accuracy: 0.9995

9/9 [==============================] - 1s 58ms/step - loss: 0.2946 - accuracy: 0.9995
Epoch 2/100
8/9 [=========================>....] - ETA: 0s - loss: 0.2992 - accuracy: 0.9956

9/9 [==============================] - 0s 54ms/step - loss: 0.2991 - accuracy: 0.9958
Epoch 3/100
9/9 [==============================] - ETA: 0s - loss: 0.2960 - accuracy: 0.9977

9/9 [==============================] - 0s 53ms/step - loss: 0.2960 - accuracy: 0.9977
Epoch 4/100
8/9 [=========================>....] - ETA: 0s - loss: 0.2953 - accuracy: 0.9985

9/9 [==============================] - 1s 57ms/step - loss: 0.2953 - accuracy: 0.9986
Epoch 5/100
9/9 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 1.0000

...

9/9 [==============================] - 1s 62ms/step - loss: 0.2922 - accuracy: 1.0000
Epoch 98/100
9/9 [==============================] - ETA: 0s - loss: 0.2922 - accuracy: 1.0000

9/9 [==============================] - 0s 54ms/step - loss: 0.2922 - accuracy: 1.0000
Epoch 99/100
9/9 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 0.9991

9/9 [==============================] - 0s 54ms/step - loss: 0.2936 - accuracy: 0.9991
Epoch 100/100
8/9 [=========================>....] - ETA: 0s - loss: 0.2984 - accuracy: 0.9971

9/9 [==============================] - 0s 52ms/step - loss: 0.2985 - accuracy: 0.9967
Epoch 1/100


ValueError: ignored

In [72]:
transitionxtrain,transitionxtest,transitionytrain,transitionytest,transitionlabels = splitData(xtrain,ytrain,'transition')
transitionxtrain.shape,transitionxtest.shape,transitionytrain.shape,transitionytest.shape,transitionlabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [73]:
transitionxtrain = transitionxtrain.reshape(4279,128,6)
transitionxtest = transitionxtest.reshape(1835,128,6)

In [82]:
temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(transitionlabels),
                                                 y = transitionlabels.ravel())
class_weights3 = {j : temp_weights[j] for j in range(len(temp_weights))}

optimizer3 = tf.keras.optimizers.Adam(learningRate)
transition_classifier = model.HART((128,6),6)
transition_classifier.compile(
    optimizer=optimizer3,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [83]:
history3 = transition_classifier.fit(
    x=transitionxtrain,
    y=transitionytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights3,
    callbacks=[checkpoint_callback],
)

Epoch 1/100
17/17 [==============================] - ETA: 0s - loss: 1.3716 - accuracy: 0.4520

17/17 [==============================] - 41s 66ms/step - loss: 1.3716 - accuracy: 0.4520
Epoch 2/100
17/17 [==============================] - ETA: 0s - loss: 0.9885 - accuracy: 0.6831

17/17 [==============================] - 1s 53ms/step - loss: 0.9885 - accuracy: 0.6831
Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8858 - accuracy: 0.7346

17/17 [==============================] - 1s 51ms/step - loss: 0.8849 - accuracy: 0.7357
Epoch 4/100
16/17 [===========================>..] - ETA: 0s - loss: 0.8321 - accuracy: 0.7695

17/17 [==============================] - 1s 54ms/step - loss: 0.8323 - accuracy: 0.7700
Epoch 5/100
17/17 [==============================] - ETA: 0s - loss: 0.7906 - accuracy: 0.7920

...

17/17 [==============================] - 2s 108ms/step - loss: 0.4279 - accuracy: 0.9986
Epoch 97/100
17/17 [==============================] - ETA: 0s - loss: 0.4280 - accuracy: 0.9984

17/17 [==============================] - 2s 111ms/step - loss: 0.4280 - accuracy: 0.9984
Epoch 98/100
17/17 [==============================] - ETA: 0s - loss: 0.4266 - accuracy: 0.9993

17/17 [==============================] - 2s 118ms/step - loss: 0.4266 - accuracy: 0.9993
Epoch 99/100
17/17 [==============================] - ETA: 0s - loss: 0.4263 - accuracy: 0.9998

17/17 [==============================] - 1s 83ms/step - loss: 0.4263 - accuracy: 0.9998
Epoch 100/100
17/17 [==============================] - ETA: 0s - loss: 0.4293 - accuracy: 0.9984

17/17 [==============================] - 2s 108ms/step - loss: 0.4293 - accuracy: 0.9984


In [84]:
transscore = transition_classifier.evaluate(transitionxtest, transitionytest)
print("transition transformer model Accuracy: ", transscore[1])

58/58 [==============================] - 3s 22ms/step - loss: 0.4264 - accuracy: 0.9995
transition transformer model Accuracy:  0.9994550347328186


In [74]:
yexpect = np.concatenate([dynamiclabels,staticlabels, transitionlabels], axis=0)

yexpect.shape

(3671,)

In [75]:
merge_arr = np.concatenate([dynamicxtest, staticxtest, transitionxtest], axis=0)


In [76]:
merge_arr.shape

(3671, 128, 6)

In [79]:
X_test.shape,Y_test.shape

((3671, 48), (3671,))

In [85]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,6)
    if(yexpect[i] in [1,2,3]):
        predict_x=dynamic_classifier.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(yexpect[i] in [4,5,6]):

        s = rf.predict(X_test[i].reshape(1, -1))  #x2test.loc[0,:].to_numpy().reshape(1,-1)
        scores.append(s[0])
    else:

        predict_x=transition_classifier.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])



    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
712
1/1 [==============================] - 0s 31ms/step
713
1/1 [==============================] - 0s 28ms/step
714
1/1 [==============================] - 0s 28ms/step
715
1/1 [==============================] - 0s 30ms/step
716
1/1 [==============================] - 0s 27ms/step
717
1/1 [==============================] - 0s 33ms/step
718
1/1 [==============================] - 0s 31ms/step
719
1/1 [==============================] - 0s 28ms/step
720
1/1 [==============================] - 0s 28ms/step
721
1/1 [==============================] - 0s 36ms/step
722
1/1 [==============================] - 0s 28ms/step
723
1/1 [==============================] - 0s 32ms/step
724
1/1 [==============================] - 0s 30ms/step
725
1/1 [==============================] - 0s 28ms/step
726
1/1 [==============================] - 0s 31ms/step
727
1/1 [==============================] - 0s 34ms/step
728
1/1 [==============================] - 0s 32ms/st

In [86]:
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))
# (static rf)(dynamic transformer)(transition rf) :  98.120%
# (static rf)(dynamic transformer)(transition transformer):  98.611 %

Accuracy score TEST: 98.611


**end** **transformers**

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(dynamicxtrain[0])
input_dim = len(dynamicxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(dynamicxtrain))



128
6
2139


In [ ]:
epochs = 100
batch_size = 16
n_hidden = 64
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model1.add(Dropout(0.5))
model1.add(LSTM(32))
model1.add(Dropout(0.5))
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18176     
                                                                 
 dropout (Dropout)           (None, 128, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 30,691
Trainable params: 30,691
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model1.fit(dynamicxtrain,
          dynamicytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

Epoch 1/100
134/134 [==============================] - 15s 17ms/step - loss: 0.8571 - accuracy: 0.5853
Epoch 2/100
134/134 [==============================] - 3s 20ms/step - loss: 0.6287 - accuracy: 0.7340
Epoch 3/100
134/134 [==============================] - 2s 18ms/step - loss: 0.4017 - accuracy: 0.8668
Epoch 4/100
134/134 [==============================] - 2s 14ms/step - loss: 0.2201 - accuracy: 0.9378
Epoch 5/100
134/134 [==============================] - 1s 11ms/step - loss: 0.2121 - accuracy: 0.9416
Epoch 6/100
134/134 [==============================] - 1s 11ms/step - loss: 0.1652 - accuracy: 0.9575
Epoch 7/100
134/134 [==============================] - 2s 14ms/step - loss: 0.1501 - accuracy: 0.9654
Epoch 8/100
134/134 [==============================] - 2s 12ms/step - loss: 0.1799 - accuracy: 0.9603
Epoch 9/100
134/134 [==============================] - 3s 20ms/step - loss: 0.1394 - accuracy: 0.9640
Epoch 10/100
134/134 [==============================] - 2s 18ms/step - loss: 0.16

In [ ]:
predict_x=model1.predict(dynamicxtest)
ypred=np.argmax(predict_x,axis=1)

29/29 [==============================] - 1s 5ms/step


In [ ]:
ypred.shape, dynamiclabels.shape

((918,), (918,))

In [ ]:

ypred[ypred == 2] = 3
ypred[ypred == 1] = 2
ypred[ypred == 0] = 1
ypred

array([3, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 3, 3, 3, 1, 1, 1, 3, 1, 3, 3, 2,
       1, 1, 1, 2, 2, 3, 2, 3, 3, 2, 3, 1, 2, 2, 1, 1, 3, 2, 1, 3, 3, 1,
       2, 1, 2, 3, 1, 1, 3, 2, 3, 2, 2, 3, 1, 3, 1, 3, 3, 1, 2, 2, 3, 3,
       2, 3, 2, 1, 2, 2, 3, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 3, 1, 3, 2, 1,
       2, 3, 3, 3, 3, 3, 2, 1, 3, 1, 3, 1, 2, 2, 1, 2, 2, 1, 1, 3, 2, 2,
       3, 1, 1, 2, 1, 2, 3, 3, 3, 3, 3, 1, 1, 3, 2, 2, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 2, 3, 1, 2, 2, 1, 2, 3, 2, 3, 3, 2, 1, 3, 2, 2, 1, 2, 3,
       1, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 1, 1, 1, 1, 2, 3, 1, 2, 3, 1, 3,
       2, 2, 3, 2, 1, 3, 3, 1, 3, 2, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 1, 2,
       1, 3, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 3, 3, 1,
       2, 3, 3, 3, 1, 2, 1, 3, 3, 3, 3, 3, 1, 2, 2, 1, 1, 1, 1, 3, 3, 3,
       1, 2, 2, 2, 3, 3, 1, 2, 1, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 1, 1, 1,
       2, 2, 3, 2, 3, 3, 1, 1, 2, 2, 1, 3, 1, 3, 2, 3, 2, 2, 3, 2, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 1, 3, 2, 1, 2, 2, 2, 1,

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, dynamiclabels)*100))

Accuracy score TEST: 99.020


**static**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(xtrain,ytrain,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [ ]:
staticxtrain =staticxtrain.reshape(2139,128,6)
staticxtest = staticxtest.reshape(918,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(staticxtrain[0])
input_dim = len(staticxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(staticxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model2.add(Dropout(0.5))
model2.add(LSTM(32))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes, activation='softmax'))
model2.summary()

128
6
2139
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 30,691
Trainable params: 30,691
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model2.fit(staticxtrain,
          staticytrain,
          batch_size=batch_size,
          #validation_data=(staticxtest, staticytest),
          epochs=epochs,
          )

Epoch 1/100
134/134 [==============================] - 5s 10ms/step - loss: 0.4238 - accuracy: 0.8125
Epoch 2/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2870 - accuracy: 0.8822
Epoch 3/100
134/134 [==============================] - 2s 13ms/step - loss: 0.3133 - accuracy: 0.8822
Epoch 4/100
134/134 [==============================] - 1s 10ms/step - loss: 0.3008 - accuracy: 0.8841
Epoch 5/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2722 - accuracy: 0.8920
Epoch 6/100
134/134 [==============================] - 1s 10ms/step - loss: 0.2893 - accuracy: 0.8925
Epoch 7/100
134/134 [==============================] - 2s 13ms/step - loss: 0.2554 - accuracy: 0.8957
Epoch 8/100
134/134 [==============================] - 2s 14ms/step - loss: 0.2848 - accuracy: 0.8976
Epoch 9/100
134/134 [==============================] - 1s 11ms/step - loss: 0.2457 - accuracy: 0.9018
Epoch 10/100
134/134 [==============================] - 1s 10ms/step - loss: 0.253

In [ ]:
predict_x=model2.predict(staticxtest)
ypred=np.argmax(predict_x,axis=1)


ypred[ypred == 2] = 6
ypred[ypred == 1] = 5
ypred[ypred == 0] = 4
ypred

from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, staticlabels)*100))

29/29 [==============================] - 0s 7ms/step
Accuracy score TEST: 91.612


In [ ]:
score = model2.evaluate(staticxtest, staticytest)
print("Accuracy: ", score[1])

29/29 [==============================] - 1s 9ms/step - loss: 0.2233 - accuracy: 0.9161
Accuracy:  0.9161220192909241


**primary**

In [ ]:
primaryxtrain,primaryxtest,primaryytrain,primaryytest,primarylabels = splitData(xtrain,ytrain,'primary')
primaryxtrain.shape,primaryxtest.shape,primaryytrain.shape,primaryytest.shape,primarylabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [ ]:
primaryxtrain =primaryxtrain.reshape(4279,128,6)
primaryxtest = primaryxtest.reshape(1835,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(primaryxtrain[0])
input_dim = len(primaryxtrain[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(primaryxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model0 = Sequential()
# Configuring the parameters
model0.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model0.add(Dropout(0.5))
model0.add(LSTM(32))
model0.add(Dropout(0.5))
model0.add(Dense(n_classes, activation='softmax'))
model0.summary()

128
6
4279
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 30,790
Trainable params: 30,790
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model0.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model0.fit(primaryxtrain,
          primaryytrain,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
268/268 [==============================] - 6s 10ms/step - loss: 1.2123 - accuracy: 0.4940
Epoch 2/100
268/268 [==============================] - 3s 10ms/step - loss: 0.8864 - accuracy: 0.6658
Epoch 3/100
268/268 [==============================] - 3s 12ms/step - loss: 0.6335 - accuracy: 0.7904
Epoch 4/100
268/268 [==============================] - 3s 11ms/step - loss: 0.4913 - accuracy: 0.8434
Epoch 5/100
268/268 [==============================] - 3s 10ms/step - loss: 0.4535 - accuracy: 0.8518
Epoch 6/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3716 - accuracy: 0.8827
Epoch 7/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3489 - accuracy: 0.8965
Epoch 8/100
268/268 [==============================] - 4s 14ms/step - loss: 0.4476 - accuracy: 0.8733
Epoch 9/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3018 - accuracy: 0.9149
Epoch 10/100
268/268 [==============================] - 3s 10ms/step - loss: 0.268

**transition**

In [ ]:
transitionxtrain,transitionxtest,transitionytrain,transitionytest,transitionlabels = splitData(xtrain,ytrain,'transition')
transitionxtrain.shape,transitionxtest.shape,transitionytrain.shape,transitionytest.shape,transitionlabels.shape

((4279, 128, 2, 3), (1835, 128, 2, 3), (4279, 6), (1835, 6), (1835,))

In [ ]:
transitionxtrain =transitionxtrain.reshape(4279,128,6)
transitionxtest = transitionxtest.reshape(1835,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(transitionxtrain[0])
input_dim = len(transitionxtrain[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(transitionxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model3 = Sequential()
# Configuring the parameters
model3.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model3.add(Dropout(0.5))
model3.add(LSTM(32))
model3.add(Dropout(0.5))
model3.add(Dense(n_classes, activation='softmax'))
model3.summary()

128
6
4279
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_4 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 30,790
Trainable params: 30,790
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model3.fit(transitionxtrain,
          transitionytrain,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
268/268 [==============================] - 7s 13ms/step - loss: 1.0277 - accuracy: 0.5691
Epoch 2/100
268/268 [==============================] - 3s 10ms/step - loss: 0.7277 - accuracy: 0.7018
Epoch 3/100
268/268 [==============================] - 3s 10ms/step - loss: 0.6189 - accuracy: 0.7689
Epoch 4/100
268/268 [==============================] - 3s 10ms/step - loss: 0.5205 - accuracy: 0.8074
Epoch 5/100
268/268 [==============================] - 3s 12ms/step - loss: 0.4553 - accuracy: 0.8317
Epoch 6/100
268/268 [==============================] - 3s 13ms/step - loss: 0.4423 - accuracy: 0.8500
Epoch 7/100
268/268 [==============================] - 3s 11ms/step - loss: 0.3886 - accuracy: 0.8696
Epoch 8/100
268/268 [==============================] - 3s 10ms/step - loss: 0.3626 - accuracy: 0.8726
Epoch 9/100
268/268 [==============================] - 3s 11ms/step - loss: 0.3433 - accuracy: 0.8843
Epoch 10/100
268/268 [==============================] - 4s 14ms/step - loss: 0.307

In [ ]:
predict_x=model3.predict(transitionxtest)
ypred=np.argmax(predict_x,axis=1)


ypred[ypred == 5] = 12
ypred[ypred == 4] = 11
ypred[ypred == 3] = 10
ypred[ypred == 2] = 9
ypred[ypred == 1] = 8
ypred[ypred == 0] = 7
ypred

58/58 [==============================] - 1s 7ms/step


array([12, 11, 10, ..., 12, 10, 10])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, transitionlabels)*100))

Accuracy score TEST: 93.842


**master lstm start**

In [ ]:
xtrain2,xtest2,ytrain2,ytest2,labels2 = splitData(xtrain,ytrain,'all')


In [ ]:
xtrain2.shape,xtest2.shape

((8559, 128, 2, 3), (3669, 128, 2, 3))

In [ ]:
xtrain2 = xtrain2.reshape(8559,128,6)
xtest2 = xtest2.reshape(3669,128,6)

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(xtrain2[0])
input_dim = len(xtrain2[0][0])
n_classes =12

print(timesteps)
print(input_dim)
print(len(transitionxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model0 = Sequential()
# Configuring the parameters
model0.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model0.add(Dropout(0.5))
model0.add(LSTM(32))
model0.add(Dropout(0.5))
model0.add(Dense(n_classes, activation='softmax'))
model0.summary()

128
6
4279
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 128, 64)           18176     
                                                                 
 dropout_8 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 12)                396       
                                                                 
Total params: 30,988
Trainable params: 30,988
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model0.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model0.fit(xtrain2,
          ytrain2,
          batch_size=batch_size,
          #validation_data=(transitionxtest, transitionytest),
          epochs=epochs,
          )

Epoch 1/100
535/535 [==============================] - 10s 12ms/step - loss: 1.6680 - accuracy: 0.3667
Epoch 2/100
535/535 [==============================] - 5s 10ms/step - loss: 1.1381 - accuracy: 0.5903
Epoch 3/100
535/535 [==============================] - 6s 12ms/step - loss: 0.9207 - accuracy: 0.6787
Epoch 4/100
535/535 [==============================] - 5s 10ms/step - loss: 0.7698 - accuracy: 0.7466
Epoch 5/100
535/535 [==============================] - 6s 11ms/step - loss: 0.6722 - accuracy: 0.7914
Epoch 6/100
535/535 [==============================] - 6s 12ms/step - loss: 0.6205 - accuracy: 0.8163
Epoch 7/100
535/535 [==============================] - 6s 10ms/step - loss: 0.5399 - accuracy: 0.8335
Epoch 8/100
535/535 [==============================] - 6s 12ms/step - loss: 0.5122 - accuracy: 0.8464
Epoch 9/100
535/535 [==============================] - 6s 10ms/step - loss: 0.4623 - accuracy: 0.8641
Epoch 10/100
535/535 [==============================] - 6s 12ms/step - loss: 0.42

In [ ]:
score = model0.evaluate(xtest2, ytest2)
print("Accuracy: ", score[1])

115/115 [==============================] - 2s 8ms/step - loss: 0.1004 - accuracy: 0.9717
Accuracy:  0.9716544151306152


**master lstm end**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)

xtrain,xtest,ytrain,ytest,ylabels = splitData(xtrain,ytrain,'all')
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((8559, 128, 2, 3), (8559, 12), (3669, 128, 2, 3), (3669, 12))

In [ ]:
predict_x=model1.predict(dynamicxtest)
ypred1=np.argmax(predict_x,axis=1)


ypred1[ypred1 == 2] = 3
ypred1[ypred1 == 1] = 2
ypred1[ypred1 == 0] = 1


predict_x=model2.predict(staticxtest)
ypred2=np.argmax(predict_x,axis=1)


ypred2[ypred2 == 2] = 6
ypred2[ypred2 == 1] = 5
ypred2[ypred2 == 0] = 4


predict_x=model3.predict(transitionxtest)
ypred3=np.argmax(predict_x,axis=1)


ypred3[ypred3 == 5] = 12
ypred3[ypred3 == 4] = 11
ypred3[ypred3 == 3] = 10
ypred3[ypred3 == 2] = 9
ypred3[ypred3 == 1] = 8
ypred3[ypred3 == 0] = 7


58/58 [==============================] - 0s 5ms/step


In [ ]:
ypred1.shape,ypred2.shape,ypred3.shape

((918,), (918,), (1835,))

In [ ]:
ypred = np.concatenate([ypred1, ypred2, ypred3], axis=0)
ypred.shape

(3671,)

In [ ]:
ypred

array([ 3,  2,  3, ..., 12, 10, 10])

In [ ]:
yexpect = np.concatenate([dynamiclabels,staticlabels, transitionlabels], axis=0)

yexpect.shape

(3671,)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred, yexpect)*100))

Accuracy score TEST: 96.949


In [ ]:
merge_arr = np.concatenate([dynamicxtest, staticxtest, transitionxtest], axis=0)


In [ ]:
merge_arr.shape

(3671, 128, 6)

In [ ]:
predict_x=model0.predict(merge_arr)
ypred3=np.argmax(predict_x,axis=1)


ypred3[ypred3 == 11] = 12
ypred3[ypred3 == 10] = 11
ypred3[ypred3 == 9] = 10
ypred3[ypred3 == 8] = 9
ypred3[ypred3 == 7] = 8
ypred3[ypred3 == 6] = 7
ypred3[ypred3 == 5] = 6
ypred3[ypred3 == 4] = 5
ypred3[ypred3 == 3] = 4
ypred3[ypred3 == 2] = 3
ypred3[ypred3 == 1] = 2
ypred3[ypred3 == 0] = 1

115/115 [==============================] - 1s 5ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ypred3, yexpect)*100))

Accuracy score TEST: 46.391


In [ ]:
ypred3.shape,merge_arr.shape

((3671,), (3671, 128, 6))

In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,6)
    if(rf_pred[i] in [1,2,3]):
        predict_x=model1.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(rf_pred[i] in [4,5,6]):
        predict_x=model2.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    else:
        predict_x=model3.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
1171
1/1 [==============================] - 0s 20ms/step
1172
1/1 [==============================] - 0s 20ms/step
1173
1/1 [==============================] - 0s 22ms/step
1174
1/1 [==============================] - 0s 19ms/step
1175
1/1 [==============================] - 0s 22ms/step
1176
1/1 [==============================] - 0s 21ms/step
1177
1/1 [==============================] - 0s 20ms/step
1178
1/1 [==============================] - 0s 20ms/step
1179
1/1 [==============================] - 0s 21ms/step
1180
1/1 [==============================] - 0s 20ms/step
1181
1/1 [==============================] - 0s 22ms/step
1182
1/1 [==============================] - 0s 21ms/step
1183
1/1 [==============================] - 0s 27ms/step
1184
1/1 [==============================] - 0s 24ms/step
1185
1/1 [==============================] - 0s 23ms/step
1186
1/1 [==============================] - 0s 20ms/step
1187
1/1 [===========================

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))

Accuracy score TEST: 97.548


In [ ]:
yexpect,Y_test

(array([ 3,  2,  3, ..., 12, 10, 10], dtype=uint32),
 array([ 3,  2,  3, ..., 12, 10, 10], dtype=uint32))

In [ ]:
#score = model1.evaluate(dynamicxtest, dynamicytest)
#print("dynamic Accuracy: ", score[1])

#score = model2.evaluate(staticxtest, staticytest)
#print("static Accuracy: ", score[1])

score = model3.evaluate(transitionxtest, transitionytest)
print("transition Accuracy: ", score[1])

score = model0.evaluate(primaryxtest, primaryytest)
print("primary Accuracy: ", score[1])

58/58 [==============================] - 4s 9ms/step - loss: 0.0388 - accuracy: 0.9929
transition Accuracy:  0.9929155111312866
58/58 [==============================] - 1s 7ms/step - loss: 0.1104 - accuracy: 0.9515
primary Accuracy:  0.9514986276626587


In [ ]:
transitionxtest2 = xtest[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]
transitionytest2 = ytest[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]
transitionlabels2 = ylabels[(ylabels==7)|(ylabels==8)|(ylabels==9)|(ylabels==10)|(ylabels==11)|(ylabels==12)]

transitionxtest2.shape,transitionytest2.shape

((1833, 128, 6), (1833, 12))

In [ ]:
transitionxtest2 = transitionxtest2.reshape(1833,128,6)

In [ ]:
transitionxtest2.shape

(1833, 128, 6)

In [ ]:
score = model3.evaluate(transitionxtest2[:100], transitionytest2[:100])
print("transition Accuracy: ", score[1])

4/4 [==============================] - 0s 9ms/step - loss: 12.2473 - accuracy: 0.1300
transition Accuracy:  0.12999999523162842


In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
primary = np.array([1,2,3,4,5,6])
len =1833 #len(xtest)

transitions = np.array([7,8,9,10,11,12])
for i in range(len):#len(xtest)
    row = transitionxtest2[i].reshape(1,128,6)
    predict_x=model3.predict(row)
    pred=transitions[np.argmax(predict_x,axis=1)]
    scores.append(pred[0])


    print(i)


1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 21ms/step
1
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 21ms/step
3
1/1 [==============================] - 0s 23ms/step
4
1/1 [==============================] - 0s 24ms/step
5
1/1 [==============================] - 0s 23ms/step
6
1/1 [==============================] - 0s 20ms/step
7
1/1 [==============================] - 0s 28ms/step
8
1/1 [==============================] - 0s 21ms/step
9
1/1 [==============================] - 0s 22ms/step
10
1/1 [==============================] - 0s 24ms/step
11
1/1 [==============================] - 0s 24ms/step
12
1/1 [==============================] - 0s 20ms/step
13
1/1 [==============================] - 0s 20ms/step
14
1/1 [==============================] - 0s 28ms/step
15
1/1 [==============================] - 0s 22ms/step
16
1/1 [==============================] - 0s 23ms/step
17
1/1 [===============

In [ ]:
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(transitionlabels2, scores)*100))

Accuracy score TEST: 36.388


**Final**

In [ ]:
xtrain,ytrain = resampleData(originalx,originaly)

xtrain,xtest,ytrain,ytest,ylabels = splitData(xtrain,ytrain,'all')
xtrain.shape,ytrain.shape,xtest.shape,ytest.shape

((8559, 128, 2, 3), (8559, 12), (3669, 128, 2, 3), (3669, 12))

In [ ]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(xtest)):#len(xtest)
    row = xtest[i].reshape(1,128,6)
    if(ylabels[i] in [1,2,3]):
        predict_x=model1.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])
        print('dynamic classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          dynamiccount = dynamiccount+1
    elif(ylabels[i] in [4,5,6]):
        predict_x=model2.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])
        print('static classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          staticcount = staticcount+1
    else:
        predict_x=model3.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
       # pred[0] = ylabels[i]
        scores.append(pred[0])
        print('transition classifier : pred: '+str(pred[0])+' actual: '+str(ylabels[i]))
        if(pred[0]!= ylabels[i]):
          transitioncount = transitioncount+1

    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
2002
1/1 [==============================] - 0s 28ms/step
static classifier : pred: 6 actual: 6
2003
1/1 [==============================] - 0s 26ms/step
static classifier : pred: 6 actual: 6
2004
1/1 [==============================] - 0s 27ms/step
transition classifier : pred: 10 actual: 7
2005
1/1 [==============================] - 0s 24ms/step
dynamic classifier : pred: 3 actual: 3
2006
1/1 [==============================] - 0s 24ms/step
transition classifier : pred: 12 actual: 10
2007
1/1 [==============================] - 0s 27ms/step
static classifier : pred: 6 actual: 6
2008
1/1 [==============================] - 0s 24ms/step
transition classifier : pred: 10 actual: 10
2009
1/1 [==============================] - 0s 25ms/step
transition classifier : pred: 10 actual: 8
2010
1/1 [==============================] - 0s 25ms/step
dynamic classifier : pred: 1 actual: 2
2011
1/1 [==============================] - 0s 24ms/step
transition cl

In [ ]:
scores=np.array(scores)
scores.shape

(3669,)

In [ ]:
ylabels.shape

(3669,)

In [ ]:

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(ylabels, scores)*100))

Accuracy score TEST: 55.492


In [ ]:
np.unique(ylabels, return_counts=True)


(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=uint32),
 array([280, 294, 299, 315, 322, 326, 310, 322, 295, 307, 310, 289]))